<a href="https://colab.research.google.com/github/veena219/Deep_Learning_Projects/blob/main/CNN_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
_URL =  'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('/content/drive/MyDrive/Colab_Notebooks/CNN/cats_and_dogs.zip', origin=_URL, extract=True)


In [ ]:
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs.zip')
PATH

In [ ]:
PATH = r"//content//drive//MyDrive//Colab_Notebooks//CNN//cats_and_dogs.zip"
New_PATH = r"//content//drive//MyDrive//Colab_Notebooks//CNN//cats_and_dogs"
!unzip $PATH -d $New_PATH

In [ ]:
dataPath = '/content/drive/MyDrive/Colab_Notebooks/CNN/cats_and_dogs/cats_and_dogs_filtered'
dataPath

In [ ]:
! ls $dataPath

In [ ]:
train_dir = os.path.join(dataPath, 'train')
validation_dir = os.path.join(dataPath, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [ ]:
num_cat_tr = len(os.listdir(train_cats_dir))
num_dog_tr = len(os.listdir(train_dogs_dir))
num_cat_val = len(os.listdir(validation_cats_dir))
num_dog_val = len(os.listdir(validation_dogs_dir))

total_train = num_cat_tr + num_dog_tr
total_val = num_cat_val + num_dog_val

In [ ]:
print('total training cat images :',num_cat_tr )
print('total training dog images :',num_dog_tr )
print('==========================================')
print('total validation cat images :',num_cat_val )
print('total validation dog images :',num_dog_val )
print('==========================================')
print(f"total training images : {total_train}")
print(f"total validation images : {total_val}")

In [ ]:
batch_size = 128 # power of 2 ---> 64, 128, 256, 512
epoch = 100
IMG_HEIGHT = 150
IMG_WIDTH = 150 

In [ ]:
## data augmentation

In [ ]:
# Horizontal flip

train_images_horizontal = ImageDataGenerator(rescale=1./255,horizontal_flip=True)
train_images_horizontal_gen = train_images_horizontal.flow_from_directory(batch_size = batch_size,
                                                                          directory = train_dir,
                                                                          shuffle = True,
                                                                          target_size = (IMG_HEIGHT,IMG_WIDTH),
                                                                          class_mode = 'binary')

In [ ]:
# augmented_images = [train_images_horizontal_gen[0][0][0] for i in range(5)]
# plotImage(augmented_images)

In [ ]:
# 2. Rotate the image


train_images_rotation = ImageDataGenerator(rescale=1./255, rotation_range = 45)
train_images_rotation_gen = train_images_rotation.flow_from_directory(batch_size = batch_size,
                                                                          directory = train_dir,
                                                                          shuffle = True,
                                                                          target_size = (IMG_HEIGHT,IMG_WIDTH),
                                                                          class_mode = 'binary')

# augmented_images = [train_images_rotation_gen[0][0][0] for i in range(5)]
# plotImage(augmented_images)

In [ ]:
# 3. Zoom the image


train_images_zoom = ImageDataGenerator(rescale=1./255, zoom_range = 0.5)
train_images_zoom_gen = train_images_zoom.flow_from_directory(batch_size = batch_size,
                                                                          directory = train_dir,
                                                                          shuffle = True,
                                                                          target_size = (IMG_HEIGHT,IMG_WIDTH),
                                                                          class_mode = 'binary')

# augmented_images = [train_images_zoom_gen[0][0][0] for i in range(5)]
# plotImage(augmented_images)

In [ ]:
train_image_generator = ImageDataGenerator(rescale = 1./255,
                                           rotation_range = 45,
                                           zoom_range = 0.5,
                                           width_shift_range = .15,
                                           height_shift_range = 0.15) # generator for our training data

In [ ]:
train_data_gen = train_image_generator.flow_from_directory( batch_size = batch_size,
                                                           directory = train_dir,
                                                           shuffle = True,
                                                           target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode = 'binary' ) # multiclass ---> "categorical"

In [ ]:
# augmented_images = [train_data_gen[0][0][0] for i in range(10)]
# plotImage(augmented_images)

In [ ]:
validation_image_generator = ImageDataGenerator(rescale = 1./255)
validation_data_gen = validation_image_generator.flow_from_directory( batch_size = batch_size,
                                                          directory = validation_dir,
                                                          target_size = (IMG_HEIGHT, IMG_WIDTH),
                                                          class_mode = 'binary' ) # multiclass ---> "categorical"

In [ ]:
 sample_training_images, check = next(train_data_gen)

In [ ]:
len(sample_training_images)

In [ ]:
# # this function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.

def plotImage(images_arr):
   fig, axes = plt.subplots(1, 5, figsize=(20,20))
   axes = axes.flatten()
   for img, ax in zip( images_arr, axes):
     ax.imshow(img)
     ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
 plotImage(sample_training_images[4:9])

In [ ]:
# let trains
# dropout

model_new = Sequential([
    Conv2D(16,3, padding = 'same', activation = 'relu', input_shape=(IMG_HEIGHT,IMG_WIDTH,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32,3, padding = 'same', activation = 'relu'),
    MaxPooling2D(),
    Conv2D(64,3, padding = 'same', activation = 'relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation = 'relu'),
    Dense(1)
])

In [ ]:
model_new.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_new.summary()

In [ ]:
# model fit

history = model_new.fit( train_data_gen,
                    steps_per_epoch=total_train//batch_size,
                    epochs=epoch,
                    validation_data=validation_data_gen,
                    validation_steps=total_val//batch_size
                    )

In [ ]:
history.history.keys()

In [ ]:
# summarize history for accuracy

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# predict with new data

img_size = (150,150)

from PIL import Image
import numpy as np

im = Image.open("/puppy-g0f356e28a_1920.jpg")
im = im.resize(img_size)
im = np.expand_dims(im, axis=0)
im = np.array(im)
im = im/255
pred = model_new.predict([im])

if pred[0][0] >= 0.5:
  prediction = "dog"
  print("dog")
else:
  prediction = "cat"
  print("cat")